# OKX Crypto Spot Data API Client

This notebook demonstrates how to use the OKX API client to fetch cryptocurrency spot data with various parameters.

## Setup

First, make sure you have:
1. Created a `.env` file with your OKX API credentials (copy from `.env.example`)
2. Installed the required dependencies: `pip install -r requirements.txt`


In [1]:
# Import the OKX API client
from utils import OKXClient, get_crypto_spot_data, get_available_instruments, get_ticker, get_recent_data, OKXAPIError
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("OKX API Client imported successfully!")


OKX API Client imported successfully!


## Basic Usage Examples

### 1. Get Recent Data (Last 24 Hours)


In [2]:
# Debug: Test environment variables and API connection
print("=== Debugging Environment Variables ===")
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if environment variables are loaded
api_key = os.getenv('OKX_API_KEY')
api_secret = os.getenv('OKX_API_SECRET')
passphrase = os.getenv('OKX_PASSPHRASE')

print(f"API Key loaded: {'Yes' if api_key else 'No'}")
print(f"API Secret loaded: {'Yes' if api_secret else 'No'}")
print(f"Passphrase loaded: {'Yes' if passphrase else 'No'}")

if not all([api_key, api_secret, passphrase]):
    print("\n❌ Missing environment variables!")
    print("Please create a .env file with your OKX API credentials:")
    print("OKX_API_KEY=your_api_key_here")
    print("OKX_API_SECRET=your_secret_here")
    print("OKX_PASSPHRASE=your_passphrase_here")
else:
    print("\n✅ All environment variables loaded successfully")
    
    # Test basic API connection without authentication first
    print("\n=== Testing Public API Endpoint ===")
    try:
        import requests
        response = requests.get('https://www.okx.com/api/v5/public/time', timeout=10)
        if response.status_code == 200:
            print("✅ OKX API is accessible")
            print(f"Server time: {response.json()}")
        else:
            print(f"❌ API returned status code: {response.status_code}")
    except Exception as e:
        print(f"❌ Failed to connect to OKX API: {e}")
    
    # Test authenticated request
    print("\n=== Testing Authenticated Request ===")
    try:
        from utils import OKXClient
        client = OKXClient()
        
        # Try to get instruments (public endpoint that requires auth)
        response = client._make_request('GET', '/api/v5/public/instruments', params={'instType': 'SPOT'})
        if response.get('data'):
            print(f"✅ Authentication successful! Found {len(response['data'])} spot instruments")
        else:
            print("❌ Authentication failed - no data returned")
            
    except Exception as e:
        print(f"❌ Authentication error: {e}")
        print("This usually means:")
        print("1. Invalid API credentials")
        print("2. API key doesn't have required permissions")
        print("3. Incorrect passphrase")
        print("4. API key is expired or disabled")


=== Debugging Environment Variables ===
API Key loaded: Yes
API Secret loaded: Yes
Passphrase loaded: Yes

✅ All environment variables loaded successfully

=== Testing Public API Endpoint ===
✅ OKX API is accessible
Server time: {'code': '0', 'data': [{'ts': '1760011992480'}], 'msg': ''}

=== Testing Authenticated Request ===
✅ Authentication successful! Found 684 spot instruments


In [3]:
# Test the candles endpoint specifically
print("=== Testing Candles Endpoint ===")
try:
    from utils import OKXClient
    
    client = OKXClient()
    
    # Test with minimal parameters
    print("Testing candles endpoint with minimal parameters...")
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5'
    })
    
    print(f"Response code: {response.get('code', 'N/A')}")
    print(f"Response message: {response.get('msg', 'N/A')}")
    print(f"Data length: {len(response.get('data', []))}")
    
    if response.get('data'):
        print("✅ Candles endpoint working!")
        print("Sample data:")
        print(response['data'][0] if response['data'] else "No data")
    else:
        print("❌ No data returned from candles endpoint")
        print("Full response:", response)
        
except Exception as e:
    print(f"❌ Candles endpoint error: {e}")
    import traceback
    traceback.print_exc()


=== Testing Candles Endpoint ===
Testing candles endpoint with minimal parameters...
Response code: 0
Response message: 
Data length: 5
✅ Candles endpoint working!
Sample data:
['1760011200000', '122730.4', '123151.9', '122717.6', '123038.3', '75.97156453', '9343531.155919843', '9343531.155919843', '0']


In [2]:
# Test the get_recent_data function step by step
print("=== Testing get_recent_data Function ===")
try:
    from utils import get_crypto_spot_data
    from datetime import datetime, timezone
    import pandas as pd
    
    # Test with explicit parameters
    print("Testing with explicit parameters...")
    
    end_time = datetime.now(timezone.utc)
    start_time = end_time - pd.Timedelta(hours=24)
    
    print(f"Start time: {start_time}")
    print(f"End time: {end_time}")
    
    df = get_crypto_spot_data(
        symbol='BTC-USDT',
        interval='1H',
        start_time=start_time,
        end_time=end_time,
        limit=5
    )
    
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    if not df.empty:
        print("✅ get_crypto_spot_data working!")
        print("First few rows:")
        print(df.head())
    else:
        print("❌ Empty DataFrame returned")
        
except Exception as e:
    print(f"❌ get_crypto_spot_data error: {e}")
    import traceback
    traceback.print_exc()


=== Testing get_recent_data Function ===
Testing with explicit parameters...
Start time: 2025-10-08 12:26:21.550250+00:00
End time: 2025-10-09 12:26:21.550250+00:00
DataFrame shape: (0, 0)
Columns: []
❌ Empty DataFrame returned


In [7]:
# Debug: Test the exact API call that get_crypto_spot_data makes
print("=== Debugging get_crypto_spot_data API Call ===")
try:
    from utils import OKXClient
    from datetime import datetime, timezone
    import pandas as pd
    
    client = OKXClient()
    
    # Replicate the exact logic from get_crypto_spot_data
    end_time = datetime.now(timezone.utc)
    start_time = end_time - pd.Timedelta(hours=24)
    
    def to_timestamp_ms(time_input):
        if time_input is None:
            return None
        if isinstance(time_input, int):
            return time_input
        if isinstance(time_input, str):
            dt = datetime.fromisoformat(time_input.replace('Z', '+00:00'))
            return int(dt.timestamp() * 1000)
        if isinstance(time_input, datetime):
            return int(time_input.timestamp() * 1000)
        raise ValueError(f"Invalid time format: {type(time_input)}")
    
    start_ts = to_timestamp_ms(start_time)
    end_ts = to_timestamp_ms(end_time)
    
    print(f"Start timestamp: {start_ts}")
    print(f"End timestamp: {end_ts}")
    
    # Prepare request parameters (exact same as in get_crypto_spot_data)
    params = {
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5'
    }
    
    if end_ts:
        params['before'] = str(end_ts)
    if start_ts:
        params['after'] = str(start_ts)
    
    print(f"API parameters: {params}")
    
    # Make the exact same API call
    response = client._make_request('GET', '/api/v5/market/candles', params=params)
    
    print(f"Response data length: {len(response.get('data', []))}")
    if response.get('data'):
        print("Sample raw data:")
        print(response['data'][0])
    else:
        print("No data returned!")
        print("Full response:", response)
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


=== Debugging get_crypto_spot_data API Call ===
Start timestamp: 1759926033670
End timestamp: 1760012433670
API parameters: {'instId': 'BTC-USDT', 'bar': '1H', 'limit': '5', 'before': '1760012433670', 'after': '1759926033670'}
Response data length: 0
No data returned!
Full response: {'code': '0', 'msg': '', 'data': []}


In [3]:
# Test: What happens without date ranges?
print("=== Testing Without Date Ranges ===")
try:
    from utils import OKXClient
    
    client = OKXClient()
    
    # Test with minimal parameters (no date range)
    print("Testing with minimal parameters (no date range)...")
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5'
    })
    
    print(f"Response data length: {len(response.get('data', []))}")
    if response.get('data'):
        print("✅ Data returned without date range!")
        print("Sample data:")
        print(response['data'][0])
    else:
        print("❌ No data returned even without date range")
        
    # Test with just 'before' parameter (recent data)
    print("\nTesting with just 'before' parameter...")
    import time
    current_time_ms = int(time.time() * 1000)
    
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5',
        'before': str(current_time_ms)
    })
    
    print(f"Response data length: {len(response.get('data', []))}")
    if response.get('data'):
        print("✅ Data returned with 'before' parameter!")
        print("Sample data:")
        print(response['data'][0])
    else:
        print("❌ No data returned with 'before' parameter")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


=== Testing Without Date Ranges ===
Testing with minimal parameters (no date range)...
Response data length: 5
✅ Data returned without date range!
Sample data:
['1760011200000', '122730.4', '123155.3', '122717.6', '123020.6', '121.18906993', '14908329.704810168', '14908329.704810168', '0']

Testing with just 'before' parameter...
Response data length: 0
❌ No data returned with 'before' parameter


In [4]:
# Test the fixed get_crypto_spot_data function
print("=== Testing Fixed get_crypto_spot_data Function ===")
try:
    from utils import get_crypto_spot_data
    from datetime import datetime, timezone
    import pandas as pd
    
    # Test with explicit parameters
    print("Testing with explicit parameters...")
    
    end_time = datetime.now(timezone.utc)
    start_time = end_time - pd.Timedelta(hours=24)
    
    print(f"Start time: {start_time}")
    print(f"End time: {end_time}")
    
    df = get_crypto_spot_data(
        symbol='BTC-USDT',
        interval='1H',
        start_time=start_time,
        end_time=end_time,
        limit=24
    )
    
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    if not df.empty:
        print("✅ get_crypto_spot_data working!")
        print("First few rows:")
        print(df.head())
        print(f"\nData range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    else:
        print("❌ Still getting empty DataFrame")
        
    # Test get_recent_data function
    print("\n=== Testing get_recent_data Function ===")
    df_recent = get_recent_data('BTC-USDT', hours=24, interval='1H')
    print(f"get_recent_data shape: {df_recent.shape}")
    if not df_recent.empty:
        print("✅ get_recent_data working!")
        print("First few rows:")
        print(df_recent.head())
    else:
        print("❌ get_recent_data still not working")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


=== Testing Fixed get_crypto_spot_data Function ===
Testing with explicit parameters...
Start time: 2025-10-08 12:26:41.494549+00:00
End time: 2025-10-09 12:26:41.494549+00:00
DataFrame shape: (0, 0)
Columns: []
❌ Still getting empty DataFrame

=== Testing get_recent_data Function ===
get_recent_data shape: (0, 0)
❌ get_recent_data still not working


In [5]:
# RESTART KERNEL FIRST! Then run this cell
# Test: Simple approach - just get recent data without date ranges
print("=== Testing Simple Approach (No Date Ranges) ===")
try:
    from utils import OKXClient
    
    client = OKXClient()
    
    # Test 1: Just get recent data (no date parameters)
    print("Test 1: No date parameters")
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5'
    })
    
    print(f"Response data length: {len(response.get('data', []))}")
    if response.get('data'):
        print("✅ Success! Sample data:")
        print(response['data'][0])
    else:
        print("❌ No data")
        
    # Test 2: Just use 'before' parameter (current time)
    print("\nTest 2: With 'before' parameter only")
    import time
    current_time_ms = int(time.time() * 1000)
    print(f"Current time (ms): {current_time_ms}")
    
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '5',
        'before': str(current_time_ms)
    })
    
    print(f"Response data length: {len(response.get('data', []))}")
    if response.get('data'):
        print("✅ Success! Sample data:")
        print(response['data'][0])
    else:
        print("❌ No data")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


=== Testing Simple Approach (No Date Ranges) ===
Test 1: No date parameters
Response data length: 5
✅ Success! Sample data:
['1760011200000', '122730.4', '123155.3', '122717.6', '123025.9', '121.53800676', '14951254.901981763', '14951254.901981763', '0']

Test 2: With 'before' parameter only
Current time (ms): 1760012811740
Response data length: 0
❌ No data


In [12]:
# Test the fixed function (after restarting kernel)
print("=== Testing Fixed get_crypto_spot_data Function ===")
try:
    # Import fresh from the updated module
    import importlib
    import utils
    importlib.reload(utils)
    from utils import get_crypto_spot_data, get_recent_data
    
    print("Testing get_crypto_spot_data with 24-hour range...")
    
    # Test with 24-hour range (should only use 'before' parameter)
    df = get_crypto_spot_data(
        symbol='BTC-USDT',
        interval='1H',
        limit=24*7  # Just get 5 candles for testing
    )
    
    print(f"DataFrame shape: {df.shape}")
    if not df.empty:
        print("✅ SUCCESS! get_crypto_spot_data working!")
        print("First few rows:")
        print(df.head())
        print(f"\nData range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    else:
        print("❌ Still getting empty DataFrame")
        
    # Test get_recent_data function
    print("\n=== Testing get_recent_data Function ===")
    df_recent = get_recent_data('BTC-USDT', hours=24, interval='1H')
    print(f"get_recent_data shape: {df_recent.shape}")
    if not df_recent.empty:
        print("✅ SUCCESS! get_recent_data working!")
        print("First few rows:")
        print(df_recent.head())
    else:
        print("❌ get_recent_data still not working")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


=== Testing Fixed get_crypto_spot_data Function ===
Testing get_crypto_spot_data with 24-hour range...
DataFrame shape: (168, 9)
✅ SUCCESS! get_crypto_spot_data working!
First few rows:
                  timestamp      open      high       low     close  \
0 2025-10-02 13:00:00+00:00  119406.3  119766.0  118909.6  119618.3   
1 2025-10-02 14:00:00+00:00  119618.3  119793.3  118657.7  118826.0   
2 2025-10-02 15:00:00+00:00  118826.1  119920.0  118500.0  119919.9   
3 2025-10-02 16:00:00+00:00  119919.9  120313.2  119591.4  119740.1   
4 2025-10-02 17:00:00+00:00  119740.1  119958.0  119368.4  119916.2   

       volume  volume_currency  volume_currency_pair  confirm  
0  861.249730     1.028024e+08          1.028024e+08        1  
1  588.853019     7.026756e+07          7.026756e+07        1  
2  438.643483     5.233103e+07          5.233103e+07        1  
3  578.304159     6.937592e+07          6.937592e+07        1  
4  342.546200     4.095866e+07          4.095866e+07        1  

Da

In [15]:
df.timestamp

0     2025-10-02 13:00:00+00:00
1     2025-10-02 14:00:00+00:00
2     2025-10-02 15:00:00+00:00
3     2025-10-02 16:00:00+00:00
4     2025-10-02 17:00:00+00:00
                 ...           
163   2025-10-09 08:00:00+00:00
164   2025-10-09 09:00:00+00:00
165   2025-10-09 10:00:00+00:00
166   2025-10-09 11:00:00+00:00
167   2025-10-09 12:00:00+00:00
Name: timestamp, Length: 168, dtype: datetime64[ns, UTC]

In [7]:
# Quick test: Use the working API call directly
print("=== Quick Test: Direct API Call ===")
try:
    from utils import OKXClient
    import pandas as pd
    
    client = OKXClient()
    
    # Use the working API call (no date ranges)
    response = client._make_request('GET', '/api/v5/market/candles', params={
        'instId': 'BTC-USDT',
        'bar': '1H',
        'limit': '24'  # Get 24 hours of data
    })
    
    if response.get('data'):
        # Convert to DataFrame manually
        df = pd.DataFrame(response['data'], columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 
            'volume_currency', 'volume_currency_pair', 'confirm'
        ])
        
        # Convert data types
        numeric_columns = ['open', 'high', 'low', 'close', 'volume', 'volume_currency', 'volume_currency_pair']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
        df['confirm'] = df['confirm'].astype(int)
        
        print(f"✅ SUCCESS! Retrieved {len(df)} data points")
        print("First few rows:")
        print(df.head())
        print(f"\nData range: {df['timestamp'].min()} to {df['timestamp'].max()}")
        print(f"Price range: ${df['low'].min():.2f} - ${df['high'].max():.2f}")
    else:
        print("❌ No data returned")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


=== Quick Test: Direct API Call ===
✅ SUCCESS! Retrieved 24 data points
First few rows:
                  timestamp      open      high       low     close  \
0 2025-10-09 12:00:00+00:00  122730.4  123155.3  122717.6  123051.4   
1 2025-10-09 11:00:00+00:00  122172.3  122844.8  122169.7  122730.3   
2 2025-10-09 10:00:00+00:00  121814.5  122339.2  121648.7  122172.4   
3 2025-10-09 09:00:00+00:00  121264.0  121849.6  121264.0  121814.5   
4 2025-10-09 08:00:00+00:00  121900.7  121955.6  121144.1  121264.0   

       volume  volume_currency  volume_currency_pair  confirm  
0  122.193446     1.503190e+07          1.503190e+07        0  
1  190.004190     2.329446e+07          2.329446e+07        1  
2  193.526391     2.361337e+07          2.361337e+07        1  
3  170.749879     2.077483e+07          2.077483e+07        1  
4  302.760755     3.677925e+07          3.677925e+07        1  

Data range: 2025-10-08 13:00:00+00:00 to 2025-10-09 12:00:00+00:00
Price range: $121144.10 - $124200

In [8]:
# Alternative: Test without authentication (public endpoints)
print("=== Testing Public Endpoints (No Auth Required) ===")
try:
    import requests
    
    # Test public time endpoint
    print("Testing public time endpoint...")
    response = requests.get('https://www.okx.com/api/v5/public/time', timeout=10)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test public instruments endpoint
    print("\nTesting public instruments endpoint...")
    response = requests.get('https://www.okx.com/api/v5/public/instruments', 
                          params={'instType': 'SPOT'}, timeout=10)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"Found {len(data.get('data', []))} spot instruments")
        if data.get('data'):
            print("Sample instrument:", data['data'][0])
    
    # Test public candles endpoint (this might require auth)
    print("\nTesting public candles endpoint...")
    response = requests.get('https://www.okx.com/api/v5/market/candles',
                          params={'instId': 'BTC-USDT', 'bar': '1H', 'limit': '5'}, 
                          timeout=10)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.text[:200]}...")
    
except Exception as e:
    print(f"Error testing public endpoints: {e}")


=== Testing Public Endpoints (No Auth Required) ===
Testing public time endpoint...
Status: 200
Response: {'code': '0', 'data': [{'ts': '1760012836568'}], 'msg': ''}

Testing public instruments endpoint...
Status: 200
Found 684 spot instruments
Sample instrument: {'alias': '', 'auctionEndTime': '', 'baseCcy': 'USDT', 'category': '1', 'contTdSwTime': '', 'ctMult': '', 'ctType': '', 'ctVal': '', 'ctValCcy': '', 'expTime': '', 'futureSettlement': False, 'instFamily': '', 'instId': 'USDT-SGD', 'instIdCode': 189535, 'instType': 'SPOT', 'lever': '', 'listTime': '1734678000000', 'lotSz': '0.001', 'maxIcebergSz': '999999999999999.0000000000000000', 'maxLmtAmt': '20000000', 'maxLmtSz': '999999999999999', 'maxMktAmt': '71000', 'maxMktSz': '1000000', 'maxStopSz': '1000000', 'maxTriggerSz': '999999999999999.0000000000000000', 'maxTwapSz': '999999999999999.0000000000000000', 'minSz': '1', 'openType': 'fix_price', 'optType': '', 'preMktSwTime': '', 'quoteCcy': 'SGD', 'ruleType': 'normal', 'settleCcy

### 2. Get Data with Specific Date Range


In [ ]:
# Get ETH-USDT data for a specific date range
try:
    # Define date range (last 7 days)
    end_time = datetime.now()
    start_time = end_time - timedelta(days=7)
    
    df_eth = get_crypto_spot_data(
        symbol='ETH-USDT',
        interval='4H',  # 4-hour intervals
        start_time=start_time,
        end_time=end_time,
        limit=200  # Get up to 200 data points
    )
    
    print(f"Retrieved {len(df_eth)} data points for ETH-USDT")
    print(f"Date range: {df_eth['timestamp'].min()} to {df_eth['timestamp'].max()}")
    print(f"Price range: ${df_eth['low'].min():.2f} - ${df_eth['high'].max():.2f}")
    
except OKXAPIError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Error: {e}")


### 3. Get Available Trading Pairs


In [ ]:
# Get list of available spot trading pairs
try:
    instruments = get_available_instruments()
    print(f"Found {len(instruments)} spot trading pairs")
    
    # Show some popular pairs
    popular_pairs = ['BTC-USDT', 'ETH-USDT', 'BNB-USDT', 'ADA-USDT', 'SOL-USDT']
    available_popular = instruments[instruments['instId'].isin(popular_pairs)]
    
    if not available_popular.empty:
        print("\nPopular trading pairs:")
        print(available_popular[['instId', 'baseCcy', 'quoteCcy', 'minSz', 'lotSz']].to_string(index=False))
    
    # Show USDT pairs
    usdt_pairs = instruments[instruments['quoteCcy'] == 'USDT']['instId'].head(10)
    print(f"\nFirst 10 USDT pairs:")
    print(usdt_pairs.tolist())
    
except OKXAPIError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Error: {e}")


### 4. Get Current Ticker Data


In [ ]:
# Get current ticker data for multiple symbols
symbols = ['BTC-USDT', 'ETH-USDT', 'BNB-USDT']

for symbol in symbols:
    try:
        ticker = get_ticker(symbol)
        if ticker:
            print(f"\n{symbol}:")
            print(f"  Last Price: ${float(ticker['last']):,.2f}")
            print(f"  24h Change: {float(ticker['chg']):.2f} ({float(ticker['chgPx']):.2f}%)")
            print(f"  24h Volume: {float(ticker['vol']):,.2f}")
            print(f"  24h High: ${float(ticker['high']):,.2f}")
            print(f"  24h Low: ${float(ticker['low']):,.2f}")
    except OKXAPIError as e:
        print(f"API Error for {symbol}: {e}")
    except Exception as e:
        print(f"Error for {symbol}: {e}")


### 5. Data Visualization


In [ ]:
# Plot price data if we have it
try:
    if 'df_btc' in locals() and not df_btc.empty:
        plt.figure(figsize=(12, 6))
        plt.plot(df_btc['timestamp'], df_btc['close'], linewidth=2, label='BTC-USDT')
        plt.title('BTC-USDT Price (Last 24 Hours)')
        plt.xlabel('Time')
        plt.ylabel('Price (USDT)')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Volume plot
        plt.figure(figsize=(12, 4))
        plt.bar(df_btc['timestamp'], df_btc['volume'], alpha=0.7)
        plt.title('BTC-USDT Volume (Last 24 Hours)')
        plt.xlabel('Time')
        plt.ylabel('Volume')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No BTC data available for plotting")
        
except Exception as e:
    print(f"Error plotting data: {e}")


## Advanced Usage Examples

### Different Time Intervals


In [ ]:
# Compare different time intervals for the same symbol
intervals = ['1m', '5m', '1H', '1D']
symbol = 'BTC-USDT'

print(f"Comparing different intervals for {symbol}:")
print("-" * 50)

for interval in intervals:
    try:
        df = get_crypto_spot_data(
            symbol=symbol,
            interval=interval,
            limit=10  # Get last 10 candles
        )
        
        if not df.empty:
            latest_price = df['close'].iloc[-1]
            price_change = ((df['close'].iloc[-1] - df['open'].iloc[0]) / df['open'].iloc[0]) * 100
            print(f"{interval:>3}: {len(df):>2} candles, Latest: ${latest_price:>8,.2f}, Change: {price_change:>6.2f}%")
        else:
            print(f"{interval:>3}: No data available")
            
    except Exception as e:
        print(f"{interval:>3}: Error - {e}")


### Error Handling Examples


In [ ]:
# Demonstrate error handling
print("Testing error handling:")
print("-" * 30)

# Test invalid symbol
try:
    df = get_crypto_spot_data('INVALID-SYMBOL', interval='1H', limit=5)
    print("Invalid symbol test: Unexpected success")
except OKXAPIError as e:
    print(f"Invalid symbol test: API Error (expected) - {e}")
except Exception as e:
    print(f"Invalid symbol test: Other error - {e}")

# Test invalid interval
try:
    df = get_crypto_spot_data('BTC-USDT', interval='invalid', limit=5)
    print("Invalid interval test: Unexpected success")
except ValueError as e:
    print(f"Invalid interval test: ValueError (expected) - {e}")
except Exception as e:
    print(f"Invalid interval test: Other error - {e}")

# Test invalid limit
try:
    df = get_crypto_spot_data('BTC-USDT', interval='1H', limit=500)  # Max is 300
    print("Invalid limit test: Unexpected success")
except ValueError as e:
    print(f"Invalid limit test: ValueError (expected) - {e}")
except Exception as e:
    print(f"Invalid limit test: Other error - {e}")
